In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb


In [2]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
wrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)


In [3]:
editor = checklist.editor.Editor()
editor.tg

In [4]:
# editor.template((('{first_name} is a good guy', 'Who is a good guy?'), '{first_name}', '3') )

In [5]:
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [8]:
def new_pp(data):
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    pr = wrapped_pp(data)[1][:, 1]
    pp = np.zeros((pr.shape[0], 3))
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

In [9]:
suite = TestSuite()

## Aspect: Vocabulary

### MFTs

In [10]:
air_noun = ['flight', 'seat', 'pilot', 'staff', 'service', 'customer service', 'aircraft', 'plane', 'food', 'cabin crew', 'company', 'airline', 'crew']
editor.add_lexicon('air_noun', air_noun)

In [11]:
print(', '.join(editor.suggest('It was {a:bert} {air_noun}.')[:40]))

great, good, excellent, amazing, bad, terrible, incredible, awful, expensive, extraordinary, new, interesting, wonderful, nice, fantastic, real, important, awesome, unusual, American, different, poor, exceptional, beautiful, ordinary, little, impressive, special, enormous, actual, easy, big, horrible, huge, fine, lousy, perfect, international, decent, terrific


In [12]:
pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'weird', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']
neutral_adj = ['American', 'international',  'commercial', 'British', 'private', 'Italian', 'Indian', 'Australian', 'Israeli', ]
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

In [13]:
print(', '.join(editor.suggest('I really {bert} the {air_noun}.')[:200]))
# print()
# print(', '.join(editor.suggest('I {bert} the {air_noun}.')[:50]))

liked, enjoyed, like, appreciate, appreciated, enjoy, loved, love, miss, missed, hate, needed, likes, wanted, got, recommend, prefer, admired, value, need, respected, want, dislike, enjoying, respect, enjoys, admire, was, feel, liking, dig, hated, mean, preferred, underestimated, disliked, used, get, use, valued, understand, did, found, adore, helped, dug, trust, remember, noticed, about, tried, hit, had, regret, took, felt, praised, cherish, have, left, loves, understood, do, LOVE, compliment, trusted, bought, thank, treasure, applaud, support, rate, know, commend, credit, underestimate, see, supported, think, thought, all, saw, picked, believe, beat, welcome, considered, impressed, improved, met, chose, thanks, deserved, envy, blame, for, help, packed, recommended, in, follow, choose, loving, misses, is, into, lost, consider, changed, worked, meant, leave, joined, wish, regretted, made, earned, rocked, cherished, take, worth, finished, experienced, are, fancy, handled, thanked, follo

In [14]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise' ]
neutral_verb_present = ['see', 'find']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']
neutral_verb_past = ['saw', 'found']
editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

Individual words

In [15]:
test = MFT(pos_adj + pos_verb_present + pos_verb_past, labels=2)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'individual positive words', 'Vocabulary', 'TODO_DESCRIPTION')

In [16]:
test = MFT(neg_adj + neg_verb_present + neg_verb_past, labels=0)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'individual negative words', 'Vocabulary', 'TODO_DESCRIPTION')

In [17]:
test = MFT(neutral_adj + neutral_verb_present + neutral_verb_past, labels=1)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'individual neutral words', 'Vocabulary', 'TODO_DESCRIPTION')

Words in context

In [18]:
t = editor.template('{it} {air_noun} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'], labels=2, save=True)
t += editor.template('{it} {be} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=2, save=True)
t += editor.template('{i} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=2, save=True)
t += editor.template('{it} {air_noun} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{it} {be} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{i} {neg_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=0, save=True)
# equivalent to:
# test = MFT(t.data, labels=t.labels, templates=t.templates)
test = MFT(**t)
suite.add(test, 'sentiment words in context', 'Vocabulary', 'TODO_DESCRIPTION')


In [19]:
t = editor.template('{it} {air_noun} {be} {neutral_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], save=True)
t += editor.template('{it} {be} {a:neutral_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], save=True)
t += editor.template('{i} {neutral_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'neutral words in context', 'Vocabulary', 'TODO_DESCRIPTION')

### Intensifiers and reducers

In [20]:
print(' , '.join(editor.suggest('{it} {be} {a:bert} {pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])[:50]))

really , very , incredibly , pretty , absolutely , truly , quite , extremely , most , actually , exceptionally , rather , particularly , genuinely , utterly , just , especially , unbelievably , amazingly , extraordinarily , always , equally , a , historically , simply , absolute , surprisingly , entirely , super , obviously , overall , exceedingly , unusually , unexpectedly , overwhelmingly , otherwise , enormously , undeniably , certainly , almost , immensely , insanely , awfully , totally , seriously , all , frankly , extraordinary , completely , real


In [21]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'amazingly', 'especially', 'exceptionally', 'unbelievably', 'utterly', 'exceedingly', 'rather', 'totally', 'particularly']

In [22]:
print(', '.join(editor.suggest('{i} {bert} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])[:100]))

really, always, truly, also, greatly, just, all, absolutely, certainly, both, definitely, personally, actually, so, especially, still, particularly, thoroughly, never, very, I, sincerely, quite, most, simply, totally, genuinely, much, honestly, obviously, sure, have, clearly, generally, highly, seriously, too, we, REALLY, strongly, had, deeply, people, rather, completely, only, dearly, mostly, already, fully, immediately, even, and, guys, do, hugely, he, family, boys, immensely, long, often, tremendously, did, ever, extremely, many, incredibly, they, surely, literally, specifically, would, sorely, kids, secretly, everyone, probably, profoundly, desperately, instantly, again, usually, then, each, heavily, has, ultimately, who, forever, vastly, fucking, enormously, constantly, Really, kinda, students, further, almost, parents


In [23]:
intens_verb = [ 'really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 'highly', 'definitely', 'certainly', 'genuinely', 'honestly', 'strongly', 'sure', 'sincerely']

In [24]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)

In [25]:
t = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
test = DIR(t.data, monotonic_label, templates=t.templates)
suite.add(test, 'intensifiers', 'Vocabulary', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)


In [26]:
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'reasonably', 'a little', 'a bit', 'slightly']

In [27]:
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_neutral_pred)

In [28]:
t = editor.template(['{it} {air_noun} {be} {pos_adj}.', '{it} {air_noun} {be} {red} {pos_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000, save=True)
t += editor.template(['{it} {air_noun} {be} {neg_adj}.', '{it} {air_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000, save=True)
test = DIR(t.data, monotonic_label_down, templates=t.templates)
suite.add(test, 'reducers', 'Vocabulary', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)


### INVariance: change neutral words

In [29]:
neutral_words = set(
    ['.', 'the', 'The', ',', 'a', 'A', 'and', 'of', 'to', 'it', 'that', 'in',
     'this', 'for',  'you', 'there', 'or', 'an', 'by', 'about', 'flight', 'my',
     'in', 'of', 'have', 'with', 'was', 'at', 'it', 'get', 'from', 'this', 'Flight', 'plane'
    ])
forbidden = set(['No', 'no', 'Not', 'not', 'Nothing', 'nothing', 'without'] + pos_adj + neg_adj + pos_verb_present + pos_verb_past + neg_verb_present + neg_verb_past)
def change_neutral(d):
#     return d.text
    examples = []
    subs = []
    words_in = [x for x in d.capitalize().split() if x in neutral_words]
    if not words_in:
        return None
    for w in words_in:
        suggestions = [x for x in editor.suggest_replace(d, w, beam_size=5, words_and_sentences=True) if x[0] not in forbidden]
        examples.extend([x[1] for x in suggestions])
        subs.extend(['%s -> %s' % (w, x[0]) for x in suggestions])
    if examples:
        idxs = np.random.choice(len(examples), min(len(examples), 10), replace=False)
        return [examples[i] for i in idxs]#, [subs[i] for i in idxs])
# Perturb.perturb(parsed_data[:5], perturb)

In [30]:
t = Perturb.perturb(sentences, change_neutral, nsamples=500)
test = INV(t.data)
suite.add(test, 'change neutral words with BERT', 'Vocabulary', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

### Add negative phrases

In [31]:
positive = editor.template('I {pos_verb_present} you.').data
positive += editor.template('You are {pos_adj}.').data
positive += ['I would fly with you again.']
positive.remove('You are happy.')
negative = editor.template('I {neg_verb_present} you.').data
negative += editor.template('You are {neg_adj}.').data
negative += ['Never flying with you again.']
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert


In [32]:
monotonic_1 = Expect.monotonic(label=2, increasing=True, tolerance=0.1)
monotonic_1_down = Expect.monotonic(label=2, increasing=False, tolerance=0.1)
monotonic_0 = Expect.monotonic(label=0, increasing=True, tolerance=0.1)
monotonic_0_down = Expect.monotonic(label=0, increasing=False, tolerance=0.1)
# goes_up = Expect.combine_and(monotonic_1, monotonic_0_down)
# goes_down = Expect.combine_and(monotonic_1_down, monotonic_0)

In [33]:
t = Perturb.perturb(parsed_data, add_phrase_function(positive), nsamples=500)
test = DIR(t.data, monotonic_1)
# test = DIR(t.data, goes_up)
suite.add(test, 'add positive phrases', 'Vocabulary', 'TODO_DESCRIPTION', overwrite=True)
# test.run(new_pp, overwrite=True)
# test.summary(3)


In [34]:
# TODO: Check this expectation function
t = Perturb.perturb(parsed_data, add_phrase_function(negative), nsamples=500)
test = DIR(t.data, monotonic_1_down)
# test = DIR(t.data, goes_down)
suite.add(test, 'add negative phrases', 'Vocabulary', 'TODO_DESCRIPTION', overwrite=True)
# test.run(new_pp, overwrite=True)
# test.summary(3)


## Aspect: robustness
### INVariance: adding irrelevant stuff before and after.


In [35]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)

# data['sentence']

def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@airline '] + urls_and_handles
    irrelevant_after = urls_and_handles 
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets


In [36]:
t = Perturb.perturb(sentences, add_irrelevant, nsamples=500)
test = INV(t.data)
suite.add(test, 'add urls and handles', 'Robustness', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

### punctuation, contractions, typos

In [37]:
t = Perturb.perturb(parsed_data, Perturb.punctuation, nsamples=500)
test = INV(t.data)
suite.add(test, 'punctuation', 'Robustness', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

In [38]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=1)
test = INV(t.data)
suite.add(test, 'typos', 'Robustness', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

In [39]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=2)
test = INV(t.data)
suite.add(test, '2 typos', 'Robustness', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)


In [40]:
t = Perturb.perturb(sentences, Perturb.contractions, nsamples=1000)
test = INV(t.data)
suite.add(test, 'contractions', 'Robustness', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

## Aspect: NER

In [41]:
t = Perturb.perturb(parsed_data, Perturb.change_names, nsamples=1000)
test = INV(t.data)
suite.add(test, 'change names', 'NER', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

In [42]:
t = Perturb.perturb(parsed_data, Perturb.change_location, nsamples=1000)
test = INV(t.data)
suite.add(test, 'change locations', 'NER', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

In [43]:
t = Perturb.perturb(parsed_data, Perturb.change_number, nsamples=1000)
test = INV(t.data)
suite.add(test, 'change numbers', 'NER', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(3)

## Aspect: temporal awareness

In [44]:
editor.template('{neg_verb_present}').data

['hate', 'dislike', 'regret', 'abhor', 'dread', 'despise']

In [45]:
change = ['but', 'even though', 'although', '']
t = editor.template(['I used to think this airline was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} I used to think it was {neg_adj}.',
                                 'In the past I thought this airline was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} in the past I thought it was {neg_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=2)
t += editor.template(['I used to {neg_verb_present} this airline, {change} now I {pos_verb_present} it.',
                                 'I {pos_verb_present} this airline, {change} I used to {neg_verb_present} it.',
                                 'In the past I would {neg_verb_present} this airline, {change} now I {pos_verb} it.',
                                 'I {pos_verb_present} this airline, {change} in the past I would {neg_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500, save=True, labels=2)

t += editor.template(['I used to think this airline was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this airline is {neg_adj}, {change} I used to think it was {pos_adj}.',
                                 'In the past I thought this airline was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this airline is {neg_adj}, {change} in the past I thought it was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=0)
t += editor.template(['I used to {pos_verb_present} this airline, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this airline, {change} I used to {pos_verb_present} it.',
                                 'In the past I would {pos_verb_present} this airline, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this airline, {change} in the past I would {pos_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500, save=True, labels=0)
test = MFT(**t)
suite.add(test, 'used to, but now', 'Temporal', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)



used to should reduce

In [46]:
t = editor.template(['{it} {be} {a:adj} {air_noun}.', 'I used to think {it} {be} {a:adj} {air_noun}.'], it=['it', 'this', 'that'], be=['is', 'was'], adj=editor.lexicons['pos_adj'] + editor.lexicons['neg_adj'], save=True)
t += editor.template(['{i} {verb} {the} {air_noun}.', '{i} used to {verb} {the} {air_noun}.'], i=['I', 'We'], the=['this', 'that', 'the'], verb=editor.lexicons['pos_verb_present'] + editor.lexicons['neg_verb_present'], save=True)
test = DIR(t.data, monotonic_label_down, templates=t.templates)
# test = MFT(data, labels=2)
suite.add(test, '"used to" should reduce', 'Temporal', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)

### Aspect:fairness

In [47]:
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'air_noun', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb'])

In [48]:
editor.template('{a:religion_adj}').data

['a Christian',
 'a Protestant',
 'a Roman Catholic',
 'an Eastern Orthodox',
 'an Anglican',
 'a Jew',
 'an Orthodox Jew',
 'a Muslim',
 'a Sunni',
 "a Shi'a",
 'an Ahmadiyya',
 'a Buddhist',
 'a Zoroastrian',
 'a Hindu',
 'a Sikh',
 'a Shinto',
 "a Baha'i",
 'a Taoist',
 'a Confucian',
 'a Jain',
 'an Atheist',
 'an Agnostic']

In [49]:
protected = {
    'race': ['a black','a hispanic', 'a white', 'an asian'],
    'sexual': editor.template('{a:sexual_adj}').data,
    'religion': editor.template('{a:religion_adj}').data,
    'nationality': editor.template('{a:nationality}').data[:20],
}

In [50]:
for p, vals in protected.items():
    print(p)
    t = editor.template(['{male} is %s {bert}.' % r for r in vals], return_maps=False, nsamples=300, save=True)
    t += editor.template(['{female} is %s {bert}.' % r for r in vals], return_maps=False, nsamples=300, save=True)
    test = INV(t.data, threshold=0.1, templates=t.templates)
    suite.add(test, 'protected: %s' % p, 'Fairness', 'TODO_DESCRIPTION')
#     test.run(new_pp)
#     test.summary(n=3)
#     print()
#     preds = np.array(test.results.preds)
#     for i, x in enumerate(vals):
#         print('%.2f %s' % (preds[:, i].mean(), vals[i]))
#     print()
#     print()
#     print('-------------------------')

race
sexual
religion
nationality



### Aspect: Negation

Simple templates:

In [51]:
t = editor.template('{it} {air_noun} {nt} {pos_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {pos_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
t += editor.template('No one {pos_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'simple negations: negative', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)

In [52]:
t = editor.template('{it} {air_noun} {nt} {neg_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neg_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
t += editor.template('No one {neg_verb_present}s {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
suite.add(test, 'simple negations: not negative', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)


In [53]:
t = editor.template('{it} {air_noun} {nt} {neutral_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:neutral_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neutral_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'simple negations: not neutral is still neutral', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)

Different templates:

In [54]:
air_noun_it = [x for x in editor.lexicons['air_noun'] if x != 'pilot']
t = editor.template('I thought {it} {air_noun} would be {pos_adj}, but it {neg}.', air_noun=air_noun_it, neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('I thought I would {pos_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'simple negations: but I did not (negative)', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)

In [55]:
t = editor.template('I thought {it} {air_noun} would be {neg_adj}, but it {neg}.', air_noun=air_noun_it, neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('I thought I would {neg_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
# suite.add(test, 'simple negations: but I did not (not negative)', 'Negation', 'TODO_DESCRIPTION')
test.run(new_pp)
test.summary(n=3)


Predicting 2124 examples
Test cases:      2124
Fails (rate):    1794 (84.5%)

Example fails:
1.0 0.0 0.0 I thought the staff would be lousy, but it wasn't.
----
0.9 0.0 0.1 I thought the flight would be unhappy, but it was not.
----
1.0 0.0 0.0 I thought this aircraft would be nasty, but it wasn't.
----


In [56]:
t = editor.template('I thought {it} {air_noun} would be {neutral_adj}, but it {neg}.', air_noun=air_noun_it, neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('I thought I would {neutral_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'simple negations: but it was not (neutral)', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)


Harder: negation with neutral in the middle

In [57]:
new_neg = neg[:-1]
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
t = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {pos_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:pos_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {pos_verb_present} {the} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we'], the=['this', 'that', 'the'], save=True)
t.data = list(np.random.choice(t.data, 1000, replace=False))
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'negation with neutral in the middle', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)

In [58]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
t = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {neg_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neg_adj} {air_noun}.',neutral=neutral,  neg=['i don\'t think', 'i can\'t say', 'i wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neg_verb_present} {the} {air_noun}.',neutral=neutral,  neg=['i don\'t think', 'i can\'t say', 'i wouldn\'t say'], i=['I', 'we'], the=['this', 'that', 'the'], save=True)
t.data = list(np.random.choice(t.data, 1000, replace=False))
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
suite.add(test, 'negation with neutral in the middle, not negative', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)


In [59]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
t = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {neutral_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neutral_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neutral_verb_present} {the} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we'], the=['this', 'that', 'the'], save=True)
t.data = list(np.random.choice(t.data, 1000, replace=False))
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'negation with neutral in the middle, neutral', 'Negation', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)


### Aspect: SRL

my opinion is more important than others

In [60]:
change = [' but', '']
templates = ['Some people think you are {neg_adj},{change} I think you are {pos_adj}.',
             'I think you are {pos_adj},{change} some people think you are {neg_adj}.',
             'I had heard you were {neg_adj},{change} I think you are {pos_adj}.',
             'I think you are {pos_adj},{change} I had heard you were {neg_adj}.',
             ]
t = editor.template(templates, change=change, unroll=True, labels=2, save=True)
templates = ['{others} {neg_verb_present} you,{change} I {pos_verb_present} you.',
             'I {pos_verb_present} you,{change} {others} {neg_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
t += editor.template(templates, others=others, change=change, unroll=True, labels=2, save=True)

change = [' but', '']
templates = ['Some people think you are {pos_adj},{change} I think you are {neg_adj}.',
             'I think you are {neg_adj},{change} some people think you are {pos_adj}.',
             'I had heard you were {pos_adj},{change} I think you are {neg_adj}.',
             'I think you are {neg_adj},{change} I had heard you were {pos_adj}.',
             ]
t += editor.template(templates, change=change, unroll=True, labels=0, save=True)
templates = ['{others} {pos_verb_present} you,{change} I {neg_verb_present} you.',
             'I {neg_verb_present} you,{change} {others} {pos_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
t += editor.template(templates, others=others, change=change, unroll=True, labels=0, save=True)
test = MFT(**t)
suite.add(test, 'my opinion is what matters', 'SRL', 'TODO_DESCRIPTION')
# test.run(new_pp)
# test.summary(n=3)

q & a form: yes

In [61]:
t = editor.template('Do I think {it} {air_noun} {be} {pos_adj}? Yes', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=2)
t += editor.template('Do I think {it} {be} {a:pos_adj} {air_noun}? Yes', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=2)
t += editor.template('Did {i} {pos_verb_present} {the} {air_noun}? Yes', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=2)
t += editor.template('Do I think {it} {air_noun} {be} {neg_adj}? Yes', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Do I think {it} {be} {a:neg_adj} {air_noun}? Yes', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Did {i} {neg_verb_present} {the} {air_noun}? Yes', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=0)
test = MFT(**t)
suite.add(test, 'Q & A: yes', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

In [62]:
t = editor.template('Do I think {it} {air_noun} {be} {neutral_adj}? Yes', it=['that', 'this', 'the'], be=['is', 'was'], save=True)
t += editor.template('Do I think {it} {be} {a:neutral_adj} {air_noun}? Yes', it=['it', 'this', 'that'], be=['is', 'was'], save=True)
t += editor.template('Did {i} {neutral_verb_present} {the} {air_noun}? Yes', i=['I', 'we'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'Q & A: yes (neutral)', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)


In [63]:
t = editor.template('Do I think {it} {air_noun} {be} {pos_adj}? No', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Do I think {it} {be} {a:pos_adj} {air_noun}? No', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Did {i} {pos_verb_present} {the} {air_noun}? No', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=0)
t += editor.template('Do I think {it} {air_noun} {be} {neg_adj}? No', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=1)
t += editor.template('Do I think {it} {be} {a:neg_adj} {air_noun}? No', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=1)
t += editor.template('Did {i} {neg_verb_present} {the} {air_noun}? No', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=1)
allow_for_neutral = lambda x, pred, _, label, _2 : pred != 0 if label == 1 else pred == label
test = MFT(t.data, Expect.single(allow_for_neutral), labels=t.labels, templates=t.templates)
suite.add(test, 'Q & A: no', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

In [64]:
t = editor.template('Do I think {it} {air_noun} {be} {neutral_adj}? No', it=['that', 'this', 'the'], be=['is', 'was'], save=True)
t += editor.template('Do I think {it} {be} {a:neutral_adj} {air_noun}? No', it=['it', 'this', 'that'], be=['is', 'was'], save=True)
t += editor.template('Did {i} {neutral_verb_present} {the} {air_noun}? No', i=['I', 'we'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'Q & A: no (neutral)', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

In [65]:
suite.save('/home/marcotcr/tmp/sentiment_suite.pkl')

In [74]:
suite = TestSuite.from_file('/home/marcotcr/tmp/sentiment_suite.pkl')

In [66]:
suite.to_raw_file('/tmp/a', n=300, seed=1)

In [67]:
texts = open('/tmp/a', 'r').read().splitlines()

In [68]:
len(texts)

53303

In [71]:
preds, confs = new_pp(texts)
open('/tmp/b', 'w').write('\n'.join(['%d %f %f %f' % (pred, *c) for pred, c in zip(preds, confs)]))

1545786

In [73]:
suite.run_from_file('/tmp/b', file_format='pred_and_softmax', overwrite=True)
suite.summary(n=3)

Vocabulary

individual positive words
Test cases:      34
Fails (rate):    0 (0.0%)


individual negative words
Test cases:      35
Fails (rate):    0 (0.0%)


individual neutral words
Test cases:      13
Fails (rate):    13 (100.0%)

Example fails:
0.0 0.0 1.0 see
----
0.0 0.0 1.0 American
----
0.0 0.0 1.0 Italian
----


sentiment words in context
Test cases:      8658
Test cases run:  300
Fails (rate):    0 (0.0%)


neutral words in context
Test cases:      1716
Test cases run:  300
Fails (rate):    287 (95.7%)

Example fails:
0.0 0.0 1.0 That pilot was American.
----
0.0 0.0 1.0 That plane is Australian.
----
0.1 0.0 0.9 It was an American aircraft.
----


intensifiers
Test cases:      2000
Test cases run:  300
Fails (rate):    0 (0.0%)


reducers
Test cases:      2000
Test cases run:  300
After filtering: 0 (0.0%)


change neutral words with BERT
Test cases:      500
Test cases run:  300
Fails (rate):    37 (12.3%)

Example fails:
0.0 0.0 1.0 @JetBlue Something to think about when 

In [74]:
# suite.run_from_file('/tmp/azure', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [75]:
# suite.run_from_file('/tmp/google', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [76]:

# suite.run_from_file('/tmp/amazon', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [77]:
# suite.run(new_pp, n=300, overwrite=True)

In [78]:
# suite.summary(n=3)